In [1]:
# Use NLTK’s Wordnet to find the meanings of words
import nltk

nltk.download('wordnet')
from nltk.corpus import wordnet as wn

def get_lemma(word):
    lemma = wn.morphy(word)
    if lemma is None:
        return word
    else:
        return lemma

from nltk.stem.wordnet import WordNetLemmatizer
def get_lemma2(word):
    return WordNetLemmatizer().lemmatize(word)

# Filter out stop words
nltk.download('stopwords')
en_stop = set(nltk.corpus.stopwords.words('english'))

# Text cleaning function
import spacy
from spacy.lang.en import English

parser = English()
def tokenize(text):
    lda_tokens = []
    tokens = parser(text)
    for token in tokens:
        if token.orth_.isspace():
            continue
        elif token.like_url:
            lda_tokens.append('URL')
        elif token.orth_.startswith('@'):
            lda_tokens.append('SCREEN_NAME')
        else:
            lda_tokens.append(token.lower_)
    return lda_tokens


# Function to prepare the text for topic modelling
def prepare_text_for_lda(text):
    tokens = tokenize(text)
    tokens = [token for token in tokens if len(token) > 4]
    tokens = [token for token in tokens if token not in en_stop]
    tokens = [get_lemma(token) for token in tokens]
    return tokens

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ibaranov\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ibaranov\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
import os
from os import listdir
from os.path import isfile, join
import pandas as pd
from collections import Counter

# directories
dir_path = os.getcwd()
#print('Working dir: ' + dir_path)

local_path = dir_path + '\\..\\gta-news\\doc2vec\data\\'
df = pd.read_pickle(local_path+'backup'+'-gta.50'+'.pickle')

# Global cluster descriptions
daily_clusters = df.groupby(['cluster'])['title']
skip_terms =['toronto','canada','canadian','ontario']
cluster_descr = []
for cluster, titles in daily_clusters:
    #print("\nCluster: ", cluster)
    filtered_words = []
    for title in titles:
        t = title[0:-4]
        #print(">>>", t)
        tokens = prepare_text_for_lda(t)
        tokens = [word for word in tokens if word not in skip_terms and not word.isdigit()]
        #print("  >", tokens)
        filtered_words = filtered_words + tokens
    count = Counter(filtered_words)
    current_clust_descr = count.most_common()[:6] 
    cluster_descr.append(current_clust_descr)
    print ("cluster " + str(cluster) + str(current_clust_descr))

# Daily cluster headers
daily_clusters = df.groupby(['date','cluster'])['title']

for name, group in daily_clusters:
    print("\nDate: ", name[0])
    print("Cluster " + str(name[1]) + " " + str(cluster_descr[name[1]]))
    for title in group:
        t = title[0:-4]
        print(">>> ", t)

cluster 0[('announce', 132), ('cannabis', 45), ('board', 29), ('technology', 23), ('update', 23), ('result', 22)]
cluster 1[('refugee', 52), ('asylum', 47), ('snowden', 46), ('shelter', 44), ('grant', 30), ('woman', 30)]
cluster 2[('announce', 68), ('project', 39), ('resource', 36), ('result', 27), ('update', 21), ('mining', 21)]
cluster 3[('weather', 23), ('winter', 22), ('storm', 16), ('school', 11), ('continue', 10), ('sweep', 9)]
cluster 4[('court', 42), ('decision', 18), ('appeal', 17), ('supreme', 16), ('government', 15), ('action', 14)]
cluster 5[('cannabis', 156), ('company', 37), ('store', 24), ('rogen', 22), ('marijuana', 20), ('growth', 20)]
cluster 6[('woman', 22), ('survivor', 21), ('family', 15), ('cancer', 14), ('holocaust', 13), ('funeral', 12)]
cluster 7[('flight', 65), ('boeing', 53), ('plane', 39), ('airline', 38), ('airport', 34), ('passenger', 32)]
cluster 8[('research', 21), ('company', 20), ('university', 20), ('better', 16), ('driving', 13), ('innovation', 13)]


Cluster 6 [('woman', 22), ('survivor', 21), ('family', 15), ('cancer', 14), ('holocaust', 13), ('funeral', 12)]
>>>  "family of 14 loses 'everything' after massive fire guts toronto home
>>>  kingston news january 21, 2019

Date:  2019-01-21 00:00:00
Cluster 8 [('research', 21), ('company', 20), ('university', 20), ('better', 16), ('driving', 13), ('innovation', 13)]
>>>  "element solidifies its spot as canada's flag bearer in the ai game
>>>  north reflects on retail lessons learned before expanding focals locations in 2019

Date:  2019-01-21 00:00:00
Cluster 9 [('study', 20), ('award', 17), ('could', 16), ('researcher', 15), ('science', 12), ('prize', 11)]
>>>  this week on the space show
>>>  microplastic pilot project looking for parry sound washing machines
>>>  men and women remember pain differently

Date:  2019-01-21 00:00:00
Cluster 10 [('crash', 71), ('ethiopian', 63), ('plane', 44), ('airline', 39), ('family', 33), ('victim', 26)]
>>>  ernest tucker (1932-2019)
>>>  yasin os

>>>  "ford says liberals' carbon tax will plunge canada into recession
>>>  doug ford touts his accomplishments and scuttles carbon tax
>>>  trudeau asks for calm at liberal caucus meeting as troubles swirl at home and abroad
>>>  carbon tax will plunge canada into recession: ford
>>>  scheer urged to offer ‘positive message’ on climate change at town hall

Date:  2019-01-21 00:00:00
Cluster 31 [('restaurant', 27), ('store', 11), ('first', 8), ('announce', 8), ('years', 7), ('design', 7)]
>>>  "new reese's pieces peanut candy launches to the edge of space to signal the first innovation to the iconic candy in its 40-year history in canada
>>>  statement by minister rodriguez on lincoln alexander day

Date:  2019-01-21 00:00:00
Cluster 32 [('resident', 18), ('downtown', 17), ('building', 16), ('power', 12), ('highrise', 11), ('homeless', 11)]
>>>  crews battle two-alarm house fire in malvern
>>>  why do i let my kid take the school bus without a seatbelt?

Date:  2019-01-21 00:00:00
Clus

>>>  "a small amount of eastern ontario municipalities to opt out of province's cannabis plans
>>>  more than half of ontario municipalities opt in as willing cannabis store hosts

Date:  2019-01-22 00:00:00
Cluster 6 [('woman', 22), ('survivor', 21), ('family', 15), ('cancer', 14), ('holocaust', 13), ('funeral', 12)]
>>>  engel & völkers hold charitable events across the country

Date:  2019-01-22 00:00:00
Cluster 7 [('flight', 65), ('boeing', 53), ('plane', 39), ('airline', 38), ('airport', 34), ('passenger', 32)]
>>>  london airport lands direct flights to three cities with discount airline
>>>  swiss railways will not take new bombardier trains until earlier problems fixed

Date:  2019-01-22 00:00:00
Cluster 8 [('research', 21), ('company', 20), ('university', 20), ('better', 16), ('driving', 13), ('innovation', 13)]
>>>  canada continues to be strong market for avaya as company transforms itself
>>>  luminari raises $845,000 in seed round funding for cpa career platform
>>>  trans

>>>  toronto cop pleads guilty to discreditable conduct after being cleared of sex assault

Date:  2019-01-22 00:00:00
Cluster 30 [('trudeau', 67), ('liberal', 55), ('carbon', 53), ('climate', 34), ('election', 33), ('government', 33)]
>>>  achampong launches legal challenge against seat slashing legislation
>>>  ford ratchets up carbon tax rhetoric
>>>  jagmeet singh: if renters spend more than 30% of their income on rent, there should be a subsidy
>>>  ontario premier defends carbon tax recession claims in wake of criticism
>>>  "'a total economic disaster': doug ford says the liberals' carbon tax will plunge canada into recession
>>>  doug ford defends carbon tax recession claims in wake of criticism
>>>  "economists cool to doug ford's warning of 'carbon tax recession'
>>>  ontario premier defends carbon tax recession claims in wake of criticism
>>>  colin mackay: about fake news
>>>  ndp, liberals pitch for urban votes with competing affordable housing plans
>>>  ontario premier d

...

>>>  toronto police recover assault-style rifle in etobicoke gun and drugs raids

Date:  2019-04-15 00:00:00
Cluster 23 [('meghan', 108), ('markle', 75), ('harry', 39), ('prince', 30), ('commonwealth', 19), ('duchess', 18)]
>>>  britons bet on new royal baby name — and diana is top of the pack
>>>  "royal fan reveals how she perfectly recreates duchess of sussex's looks for a fraction of the price
>>>  all the times meghan rocked canadian brands during her pregnancy

Date:  2019-04-15 00:00:00
Cluster 25 [('housing', 51), ('estate', 24), ('mortgage', 17), ('stress', 14), ('affordable', 14), ('money', 12)]
>>>  "high-end hostels offer 'flashpackers' affordable luxuries, social connection
>>>  bitcoin transaction costs are ‘so much cheaper’ than credit cards, says canadian mayor
>>>  a perfect storm

Date:  2019-04-15 00:00:00
Cluster 26 [('story', 13), ('holocaust', 11), ('artist', 11), ('exhibition', 10), ('history', 9), ('notre', 9)]
>>>  notre dame cathedral fire: historian explains 

Cluster 11 [('health', 38), ('measles', 27), ('vaccine', 24), ('study', 23), ('cancer', 22), ('treatment', 22)]
>>>  study finds peanut allergy treatment safe for allergists to use with young kids
>>>  "sickkids shuts down motherisk helplines after grants, donations 'reduced to zero'
>>>  social support key to remission from suicidal thoughts for disabling chronic pain sufferers
>>>  study finds peanut allergy treatment safe for allergists to use with young kids

Date:  2019-04-17 00:00:00
Cluster 12 [('health', 96), ('government', 46), ('budget', 35), ('funding', 35), ('public', 26), ('province', 23)]
>>>  cost of federal carbon tax will burden ontario’s police, firefighters, corrections and other frontline services
>>>  city to stay at table with province while awaiting answers on transit plan
>>>  ontario to allow social housing providers to ban tenants evicted for criminal activity
>>>  "ford government promises $1b to fix ontario's affordable housing crisis
>>>  "social housing pr

>>>  a photographer ventured inside this abandoned detroit church and explored its haunting interior
>>>  the national gallery of canada presents masterpiece in focus: friedrich nietzsche and the artists of the new weimar
>>>  rebuilding notre dame: u of t expert on the history and future of the iconic cathedral
>>>  emperors and jewels: the al-sabah collection, kuwait – canadian art
>>>  agnès varda, a pioneering artist who saw the extraordinary in the ordinary
>>>  multiculturalism is not the real story of canada
>>>  conference explores democracy, freedom – the williams record
>>>  "what jordan peterson's followers don't know about žižek or postmodernism

Date:  2019-04-17 00:00:00
Cluster 27 [('barrick', 39), ('newmont', 33), ('dollar', 29), ('company', 21), ('investor', 19), ('stock', 19)]
>>>  "'i couldn't even get meetings': oil and gas companies bemoan investor apathy
>>>  aphria (apha): growing pains in the cannabis industry
>>>  "'i couldn't even get meetings': oil and gas co

>>>  manhunt underway for fake deliveryman who shot woman with crossbow
>>>  windsor police investigate shots fired near radio station

Date:  2019-04-17 00:00:00
Cluster 43 [('oscar', 40), ('netflix', 29), ('season', 28), ('award', 26), ('first', 26), ('movie', 25)]
>>>  tiff comes of age: iconic film festival turns 18
>>>  "depression gets egged in thomas mckechnie's 4½ (ig)noble truths

Date:  2019-04-17 00:00:00
Cluster 44 [('police', 53), ('mcarthur', 45), ('review', 33), ('bruce', 32), ('miss', 30), ('person', 30)]
>>>  investigated canadore official remains on the job

Date:  2019-04-17 00:00:00
Cluster 45 [('founder', 19), ('truaxe', 18), ('brandon', 18), ('deciem', 18), ('reporter', 16), ('police', 10)]
>>>  why a cold-blooded criminal spared the life of the man that killed his loved one
>>>  he built, then nearly broke, a successful beauty start-up. can it go on without him?

Date:  2019-04-17 00:00:00
Cluster 46 [('police', 140), ('charge', 137), ('arrest', 74), ('assault', 

>>>  ahead of federal election, imams at 69 canadian mosques deliver message that every vote counts
>>>  right-wing populist united conservatives win alberta election as ndp vote plummets

Date:  2019-04-20 00:00:00
Cluster 31 [('restaurant', 27), ('store', 11), ('first', 8), ('announce', 8), ('years', 7), ('design', 7)]
>>>  serea restaurant slated to open in hotel del coronado in june 2019
>>>  "canada's 100 best restaurants: serving up excellence in our nation's food scene

Date:  2019-04-20 00:00:00
Cluster 33 [('china', 122), ('huawei', 117), ('ambassador', 69), ('extradition', 47), ('mccallum', 39), ('executive', 36)]
>>>  sikh diaspora bodies express concern over indian interference in canada

Date:  2019-04-20 00:00:00
Cluster 36 [('attack', 51), ('mosque', 46), ('zealand', 40), ('muslim', 33), ('shooting', 32), ('handgun', 25)]
>>>  "competition for sikh vote: canada's quiet surrender to khalistanis
>>>  ‘a place where we can be safe’: women’s mosque of canada opens in toronto